In [2]:
import torch
from torchsummary import summary
import torch.nn as nn


In [3]:
# e.g 512,512

class Downsample(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels=32,
                #  internal_ratio=4,
                 kernel_size=3,
                 padding=0,
                 dropout_prob=0.,
                 stride = 2,
                 bias=False,
                 relu=True,
                 out_channels_div= [11,9,9] 
            ):
        super(Downsample,self).__init__()


        if relu:
            activation = nn.ReLU()
        else:
            activation = nn.PReLU()

        self.conv2d_d1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels_div[0],kernel_size=kernel_size, stride=stride, padding=padding, bias=bias, dilation=1)
        self.conv2d_d2 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels_div[1],kernel_size=kernel_size, stride=stride, padding=1, bias=bias, dilation=2)
        self.conv2d_d5 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels_div[2],kernel_size=kernel_size, stride=stride, padding=4, bias=bias, dilation=5)
        self.ext_branch = nn.MaxPool2d(kernel_size, stride=2, padding=padding)
        # self.half = nn.MaxPool2d(2, stride=2)
        self.activation = activation
        self.batch_norm = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        print('input:', input.shape)
        main1_d1= self.conv2d_d1(input)
        print('dialation 1:',main1_d1.shape)
        main2_d2 = self.conv2d_d2(input)
        print('dialation 2', main2_d2.shape)
        main3_d5 = self.conv2d_d5(input)
        print('dialation 5:',main3_d5.shape)
        # ext1=self.half(input)
        ext1 = self.ext_branch(input)
        print(ext1.shape)
        
        
        
        out = torch.cat((main1_d1,main2_d2,main3_d5,ext1), dim=1)
        print('out after concate', out.shape)
        out = self.batch_norm(out)
        out = self.activation(out)
        # print('final', out)


        return out



In [4]:
model = Downsample(in_channels=3)

In [5]:
x = torch.randn(1,3, 360, 640)
xx = model(x)

input: torch.Size([1, 3, 360, 640])
dialation 1: torch.Size([1, 11, 179, 319])
dialation 2 torch.Size([1, 9, 179, 319])
dialation 5: torch.Size([1, 9, 179, 319])
torch.Size([1, 3, 179, 319])
out after concate torch.Size([1, 32, 179, 319])
C:\Users\abdul\.conda\envs\torch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [6]:
print(model)

Downsample(
  (conv2d_d1): Conv2d(3, 11, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (conv2d_d2): Conv2d(3, 9, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(2, 2), bias=False)
  (conv2d_d5): Conv2d(3, 9, kernel_size=(3, 3), stride=(2, 2), padding=(4, 4), dilation=(5, 5), bias=False)
  (ext_branch): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (activation): ReLU()
  (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [7]:
# e.g 512,512

class Regular(nn.Module):
    def __init__(self,
                 in_channels=32,
                 out_channels=32,
                #  internal_ratio=4,
                 kernel_size=3,
                 padding=0,
                 dropout_prob=0.,
                 stride = 1,
                 bias=False,
                 relu=True, 
                 out_channels_div = [11,11,10]
                 
                 ):
        super(Regular,self).__init__()


        if relu:
            activation = nn.ReLU()
        else:
            activation = nn.PReLU()

        self.conv2d_d1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels_div[0],kernel_size=kernel_size, stride=stride, padding=1, bias=bias, dilation=1)
        self.conv2d_d2 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels_div[1],kernel_size=kernel_size, stride=stride, padding=2, bias=bias, dilation=2)
        self.conv2d_d5 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels_div[2],kernel_size=kernel_size, stride=stride, padding=5, bias=bias, dilation=5)
        # self.half = nn.MaxPool2d(2, stride=2)
        self.activation = activation
        self.batch_norm = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        # print('input:', input.shape)
        main1_d1= self.conv2d_d1(input)
        print('dialation 1:',main1_d1.shape)
        main2_d2 = self.conv2d_d2(input)
        print('dialation 2', main2_d2.shape)
        main3_d5 = self.conv2d_d5(input)
        print('dialation 5:',main3_d5.shape)
        # ext1=self.half(input)
        
        
        
        out = torch.cat((main1_d1,main2_d2,main3_d5), dim=1)
        print('out after concate', out.shape)
        out = self.batch_norm(out)
        out = self.activation(out)
        # print('final', out)


        return out



In [8]:
model2 = Regular(3)

In [9]:
model2(x)

dialation 1: torch.Size([1, 11, 360, 640])
dialation 2 torch.Size([1, 11, 360, 640])
dialation 5: torch.Size([1, 10, 360, 640])
out after concate torch.Size([1, 32, 360, 640])


tensor([[[[0.0000, 0.4039, 0.0000,  ..., 0.0000, 0.8166, 0.0000],
          [0.6406, 0.0000, 0.5942,  ..., 0.0038, 0.9530, 1.5304],
          [0.5986, 0.0000, 0.1506,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.8491, 0.0000, 0.0000,  ..., 0.4574, 0.0000, 0.9222],
          [0.0000, 0.7526, 0.0000,  ..., 0.0000, 0.0000, 0.6954],
          [0.0000, 0.0000, 0.3739,  ..., 0.0000, 0.3922, 0.0000]],

         [[0.0000, 1.0010, 0.1104,  ..., 0.0000, 0.6418, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 1.3113, 3.4148, 0.8746],
          [0.8499, 0.0000, 0.7296,  ..., 0.0000, 0.4518, 0.0000],
          ...,
          [1.1909, 0.0000, 0.0000,  ..., 0.6962, 0.0000, 0.9692],
          [0.6480, 1.4951, 0.0000,  ..., 0.0000, 0.9164, 0.8673],
          [0.0000, 0.0000, 1.1174,  ..., 0.0000, 0.0000, 0.8118]],

         [[0.0000, 0.4219, 1.0575,  ..., 0.1947, 0.4190, 0.0899],
          [0.0000, 0.5272, 0.9461,  ..., 0.0000, 0.3880, 0.0000],
          [0.3897, 1.1210, 2.0873,  ..., 0

In [10]:
print(model2)

Regular(
  (conv2d_d1): Conv2d(3, 11, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv2d_d2): Conv2d(3, 11, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2), bias=False)
  (conv2d_d5): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(5, 5), dilation=(5, 5), bias=False)
  (activation): ReLU()
  (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [12]:
class Encoder(nn.Module):
    def __init__(self, classes, encoder_relu=False, decoder_relu=True):
        super().__init__()
        self.downsample_1 = Downsample(in_channels=3,out_channels=32, kernel_size=3, relu=True, out_channels_div=[11,9,9])
        self.regular_1 = Regular(in_channels=32, out_channels=32, kernel_size=3, out_channels_div=[11,11,10])
        self.downsample_2 = Downsample(in_channels=32, out_channels=64, kernel_size=3,relu=True, out_channels_div=[11,11,10])
        self.regular_2 = Regular(in_channels=64, out_channels=64,out_channels_div=[22,21,21])
        self.downsample_3 = Downsample(in_channels=64, out_channels=128, out_channels_div=[22,21,21])
        self.regular_3 = Regular(in_channels=128, out_channels=128, out_channels_div=[43,43,42])
        self.regular_4 = Regular(in_channels=128, out_channels=128, out_channels_div=[43,43,42] )

    


    def forward(self, input):
        x = self.downsample_1(input)
        x = self.regular_1(x)
        x = self.downsample_2(x)
        x = self.regular_2(x)
        x = self.downsample_3(x)
        x= self.regular_3(x)
        x= self.regular_4(x)

        return x

        


In [13]:
encoder = Encoder(3)

In [14]:
out = encoder(x)
print(out)

input: torch.Size([1, 3, 360, 640])
dialation 1: torch.Size([1, 11, 179, 319])
dialation 2 torch.Size([1, 9, 179, 319])
dialation 5: torch.Size([1, 9, 179, 319])
torch.Size([1, 3, 179, 319])
out after concate torch.Size([1, 32, 179, 319])
dialation 1: torch.Size([1, 11, 179, 319])
dialation 2 torch.Size([1, 11, 179, 319])
dialation 5: torch.Size([1, 10, 179, 319])
out after concate torch.Size([1, 32, 179, 319])
input: torch.Size([1, 32, 179, 319])
dialation 1: torch.Size([1, 11, 89, 159])
dialation 2 torch.Size([1, 11, 89, 159])
dialation 5: torch.Size([1, 10, 89, 159])
torch.Size([1, 32, 89, 159])
out after concate torch.Size([1, 64, 89, 159])
dialation 1: torch.Size([1, 22, 89, 159])
dialation 2 torch.Size([1, 21, 89, 159])
dialation 5: torch.Size([1, 21, 89, 159])
out after concate torch.Size([1, 64, 89, 159])
input: torch.Size([1, 64, 89, 159])
dialation 1: torch.Size([1, 22, 44, 79])
dialation 2 torch.Size([1, 21, 44, 79])
dialation 5: torch.Size([1, 21, 44, 79])
torch.Size([1, 64

In [15]:
print(encoder)

Encoder(
  (downsample_1): Downsample(
    (conv2d_d1): Conv2d(3, 11, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (conv2d_d2): Conv2d(3, 9, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(2, 2), bias=False)
    (conv2d_d5): Conv2d(3, 9, kernel_size=(3, 3), stride=(2, 2), padding=(4, 4), dilation=(5, 5), bias=False)
    (ext_branch): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (activation): ReLU()
    (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (regular_1): Regular(
    (conv2d_d1): Conv2d(32, 11, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2d_d2): Conv2d(32, 11, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2), bias=False)
    (conv2d_d5): Conv2d(32, 10, kernel_size=(3, 3), stride=(1, 1), padding=(5, 5), dilation=(5, 5), bias=False)
    (activation): ReLU()
    (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, tra

In [16]:
class Upsample(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size = 3, relu = True):
        super(Upsample,self).__init__()

        if relu:
            activation = nn.ReLU
        else:
            activation = nn.PReLU


        
        
        self.conv = nn.ConvTranspose2d(in_channels= in_channels, 
                                        out_channels=out_channels, stride=2,
                                        kernel_size=kernel_size, 
                                        padding=0, output_padding=0,
                                        bias=True
                                        )
        # self.bn = nn.BatchNorm2d(noutput, eps=1e-3)     # ERFNet batchnorm with eps = e-5
        self.batch_norm = nn.BatchNorm2d(out_channels)
        # self.regular_de_1=Regular(in_channels=64,out_channels=64 ,out_channels_div=[22,21,21])
        self.activation = activation
    def forward(self, input):
        print('Input Shape: ', input.shape)
        output = self.conv(input)
        output = self.batch_norm(output)
        print('Output Shape: ', output.shape)
        output = self.activation(output)
        
        # output = self.regular_de_1(output)
        # output = nn.ReLU(output)
        return output

In [17]:
test = Upsample(in_channels=128, out_channels=64) 

In [18]:
test

Upsample(
  (conv): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2))
  (batch_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [28]:
t= torch.rand(1,128,44,79)

In [29]:
yy =test(t)

Input Shape:  torch.Size([1, 128, 44, 79])
Output Shape:  torch.Size([1, 64, 89, 159])


In [21]:
# e.g 512,512

class Regular_decode(nn.Module):
    def __init__(self,
                 in_channels=32,
                 out_channels=32,
                #  internal_ratio=4,
                 kernel_size=3,
                 padding=0,
                 dropout_prob=0.,
                 stride = 1,
                 bias=False,
                 relu=True, 
                 out_channels_div = [11,11,10]
                 
                 ):
        super(Regular_decode,self).__init__()


        if relu:
            activation = nn.ReLU()
        else:
            activation = nn.PReLU()

        self.conv2d_d1 = nn.Conv2d(in_channels=in_channels, 
                                    out_channels=out_channels_div[0],
                                    kernel_size=kernel_size, 
                                    stride=stride, 
                                    padding=1, 
                                    bias=bias, 
                                    dilation=1
                                    )

        self.conv2d_d2 = nn.Conv2d(in_channels=in_channels, 
                                    out_channels=out_channels_div[1],
                                    kernel_size=kernel_size, 
                                    stride=stride, 
                                    padding=2, 
                                    bias=bias, 
                                    dilation=2
                                    )

        self.conv2d_d5 = nn.Conv2d(in_channels=in_channels, 
                                    out_channels=out_channels_div[2],
                                    kernel_size=kernel_size, 
                                    stride=stride, padding=5, 
                                    bias=bias, 
                                    dilation=5
                                    )

        # self.half = nn.MaxPool2d(2, stride=2)

        self.activation = activation
        
        self.batch_norm = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        # print('input:', input.shape)
        main1_d1= self.conv2d_d1(input)
        print('dialation 1:',main1_d1.shape)
        main2_d2 = self.conv2d_d2(input)
        print('dialation 2', main2_d2.shape)
        main3_d5 = self.conv2d_d5(input)
        print('dialation 5:',main3_d5.shape)
        # ext1=self.half(input)
        
        
        
        out = torch.cat((main1_d1,main2_d2,main3_d5), dim=1)
        print('out after concate', out.shape)
        out = self.batch_norm(out)
        out = self.activation(out)
        # print('final', out)


        return out



In [30]:
test_1 = Regular_decode(in_channels=128, out_channels=64 ,out_channels_div=[22,21,21])

In [31]:
t= torch.rand(1,128,89,159)

In [32]:
test_1(t)

dialation 1: torch.Size([1, 22, 89, 159])
dialation 2 torch.Size([1, 21, 89, 159])
dialation 5: torch.Size([1, 21, 89, 159])
out after concate torch.Size([1, 64, 89, 159])


tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 1.9491, 0.0000,  ..., 0.9291, 0.7268, 0.1285],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0153, 0.0000],
          ...,
          [0.0000, 0.0770, 1.1516,  ..., 0.0000, 0.4107, 0.0000],
          [0.0000, 0.0237, 0.0000,  ..., 0.0000, 0.9543, 0.0000],
          [0.2365, 0.0000, 0.0000,  ..., 0.0000, 1.5381, 0.4400]],

         [[0.0000, 0.1946, 0.3209,  ..., 1.3862, 0.3922, 0.6012],
          [0.0000, 1.1482, 0.0000,  ..., 1.1905, 0.0000, 0.9589],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4686, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.6855],
          [0.0000, 1.7284, 0.8159,  ..., 1.1612, 1.0018, 0.1172],
          [0.0000, 0.4396, 0.0000,  ..., 0.8046, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.4905],
          [0.0000, 0.0000, 0.0000,  ..., 0.5633, 0.2951, 0.4702],
          [0.0000, 0.3210, 0.0000,  ..., 0

In [33]:
class Decoder(nn.Module):
    def __init__(self, num_classes):
        super(Decoder, self).__init__()
        self.upsample_1 = Upsample(in_channels=128, out_channels=64)
        self.regular_de_1 = Regular_decode(in_channels=64, out_channels=64, out_channels_div=[22,21,21])


    def forward(self, input):
        x = self.upsample_1(input)
        x = self.regular_de_1(x)
        

        return x



        


In [34]:
model = Decoder(3)

In [35]:
rand = torch.rand(1,128,44,79)

In [36]:
rand.shape

torch.Size([1, 128, 44, 79])

In [302]:
model

Decoder(
  (upsample_1): Upsample(
    (conv): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2))
    (batch_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (regular_de_1): Regular_decode(
    (conv2d_d1): Conv2d(64, 22, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2d_d2): Conv2d(64, 21, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2), bias=False)
    (conv2d_d5): Conv2d(64, 21, kernel_size=(3, 3), stride=(1, 1), padding=(5, 5), dilation=(5, 5), bias=False)
    (activation): ReLU()
    (batch_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [37]:
ouput = model(rand)

Input Shape:  torch.Size([1, 128, 44, 79])
Output Shape:  torch.Size([1, 64, 89, 159])


TypeError: conv2d() received an invalid combination of arguments - got (ReLU, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!ReLU!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!ReLU!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)
